##ML + STRUCTURED STREAMING

#Costruzione di un modello ML e predizione in tempo reale sfruttando lo straming strutturato

Il modello creato viene poi esportato per essere usato da uno script successivo


In [ ]:
#IMPORT
import pyspark.ml.regression as rg
import pyspark as pys 
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
import pyspark.sql.functions as f
import pyspark.ml.feature as feat
import numpy as np
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import json 


import pyspark.ml.evaluation as ev

#CONFIGURAZIONE PATH

#File contente il dataset completo
forest_path = 'forest_coverage_type.csv'
#Directory in cui esportare il modello e lo schema nel file schema.json
modelPath="/home/lorenzo/Documenti/PySpark-tesi/Prototipo/exported"

In [ ]:

#Inizializzazione spark session/context e caricamento dataset

#Spark session
spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

#Spark context
sc = spark.sparkContext

#Caricamento dataset da file CSV

forest=spark.read.csv(
    forest_path,
    header=True,
    inferSchema=True
)

print("Schema DF: ")
forest.printSchema()

#Divisione fra train e test
forest_train, forest_test=(
    forest.randomSplit([0.7,0.3],seed=123)
)

#Esporto il dataset di test in formato CSV (in un unico file con repartition(1))
forest_test.repartition(1).write.csv("forest_test.csv","overwrite")

In [ ]:
#Creazione modello di ML

#Cerchiamo di predirre la colonna Elevation che è la prima nel dataset

#Usiamo una pipeline in 2 stadi per la creazione di modelli ML in PySpark

#1) Istanzione un oggetto della classe VectorAssembler
#Che per permette di fondere tutte le colonne in una
vectorAssembler = feat.VectorAssembler(
    inputCols=forest.columns[1:] #la 1 è "elevation"
    , outputCol='features'
    )

#2) Istanzione un oggetto della classe RandomForestRegressor
#Implementa l'algoritmo di regressione "Random Forest" e crea il modello ML
rf_obj = rg.RandomForestRegressor(
    labelCol='Elevation' #target value (colonna da predire)
    , maxDepth=10
    , minInstancesPerNode=10
    , minInfoGain=0.1
    , numTrees=10
    )

#Pipeline 
pip = Pipeline(stages=[vectorAssembler, rf_obj])

#Modello ML 
pModel = ( #DF come quello di input ma con le colonne features e predicition 
    pip.fit(forest_train)
)


In [ ]:
#Esportazione dati

#Salvo il modello ML ottenuto
pModel.write().overwrite().save(modelPath)

#Esporto lo schema del dataset forest

with open(modelPath+"/schema.json", "w") as f:
    json.dump(forest.schema.jsonValue(), f)

In [ ]:
#Test del modello su forest_test (solo batch)
results=(
    pModel
    .transform(forest_test)
    .select("Elevation","prediction")
)

#5 predizioni di esempio (da forest_train)
results.show(5)

In [ ]:
#Valutazione accuratezza 
evaluator = ev.RegressionEvaluator(labelCol='Elevation')
evaluator.evaluate(results, {evaluator.metricName: 'r2'})